In [11]:
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish
import copy
from IPython.display import display
import pandas as pd

In [4]:
def scoreToInt( score ):
    t = str(score) #resultAnalyse["score"].pov( colorBool )
    return round(int(t)/100.,2)

In [2]:
class UtilsEngine:

    def __init__(self ):
        self.engine = chess.engine.SimpleEngine.popen_uci("../shared/stockfish_12_win_x64/stockfish_20090216_x64")
    
    def engine(self):
        return self.engine
    
    def analyse(self, board):
        return self.engine.analyse( board , chess.engine.Limit(time=0.75)) #params ..Limit(depth=20) // time=0.75
    
    def score(self, board, colorBool= True ):
        resultAnalyse = self.analyse(board)
        return scoreToInt(resultAnalyse["score"].pov( colorBool )) # point of view score, white>True

    def showLine(self, board):
        analysis = self.engine.analysis(board)
        for info in analysis:
            print(info.get("score"), info.get("pv"))
            if info.get("seldepth", 0) > 5:
                break
    
    def top(self, board):
        infoMultiVariant = self.engine.analyse( board , chess.engine.Limit(time=0.3), multipv=3 )
        return [ f"{variant['pv'][0]}→{scoreToInt(variant['score'].pov(board.turn))}" for variant in infoMultiVariant]
    
    def topList(self, board):
        infoMultiVariant = self.engine.analyse( board , chess.engine.Limit(time=0.3), multipv=3 )
        return [ [str(variant['pv'][0])[0:2], str(variant['pv'][0])[2:4], scoreToInt(variant['score'].pov(board.turn))] for variant in infoMultiVariant]

    def typeMove(self, score1, score2, isWhite):
        movedColor = True
        dif = abs(score1-score2)
        advantageWhite = (score2 > 0)
        if isWhite is not advantageWhite:
            accuracy = ''
            if dif < 1:
                accuracy = 'inaccuracy'
            elif dif >= 1 and dif < 2:
                accuracy = 'mistake'
            else:
                accuracy = 'blunder'
            return(f'error: {accuracy}')
        else:
            return('ok')
                
    def options(self, board):
        '''
        return sorted data frame with best moves
        '''
        if board.turn: 
            print('White to move')
        else: 
            print('Black to move')
        movesData = []
        for el in board.legal_moves:
            info = self.engine.analyse(board, chess.engine.Limit(time=0.2), root_moves=[el])
            t = str(info["score"].pov(board.turn))
            if not t.startswith('#'): # if is not MATE: print(str(board.san(el))," eval = mate in ", t)
                moveRowData = [el, board.san(el), round(int(t)/100.,2) ]
                movesData.append(moveRowData)
                # print( el, str(board.san(el))," eval = ", round(int(t)/100.,2))
        dfMoves = pd.DataFrame.from_records(movesData, columns=['move_cells', 'move_san', 'score'])
        return dfMoves.sort_values(by=['score'], ascending=False)

        

In [7]:
utilsEngine = UtilsEngine()

In [28]:
def typeMove( score1, score2, isWhite):
        movedColor = True
        dif = abs(score1-score2)
        advantageWhite = (score2 > 0)
        if isWhite is not advantageWhite:
            accuracy = ''
            if dif < 1:
                accuracy = 'inaccuracy'
            elif dif >= 1 and dif < 2:
                accuracy = 'mistake'
            else:
                accuracy = 'blunder'
            return(f'error: {accuracy}')
        else:
            return('ok')
        
typeMove( 0.43, -0.71, False)

'ok'

### Usage in notebook

```
%run ../shared/util_engine.ipynb
```